In [1]:
!pip install openai

In [10]:
import os
from openai import OpenAI
from getpass import getpass

In [11]:
os.environ['OPENAI_API_KEY'] = getpass("OPENA_API_KEY")

OPENA_API_KEY··········


In [12]:
client = OpenAI()

In [26]:
# Function to call OpenAI
def get_response_v0(prompt):

    completion = client.chat.completions.create(
      model="gpt-4o",
      messages=[
          {
              "role": "user",
              "content": prompt,
          },
      ],
    )

    return completion.choices[0].message.content

## Zero Shot Classification

In [27]:
complaint = (
    "The delivery guy just left the package at the gate without ringing the bell. "
    "I found it two hours later completely soaked from the rain."
)

# Without Chain of Thought Prompt
basic_prompt = f"""
Classify the following customer complaint into one of these categories:
1. Delivery issue
2. Product quality
3. Customer service
4. Other

Complaint: "{complaint}"

Category:"""

print(get_response_v0(basic_prompt), "\n")

1. Delivery issue 



## Few Shots Classification

In [22]:
# Few-shot examples
few_shot_prompt = f"""
Classify the following customer complaints into one of these categories:
1. Delivery issue
2. Product quality
3. Customer service
4. Other

Examples:

Complaint: "My order was supposed to arrive yesterday, but it hasn’t shown up yet."
Category: Delivery issue

Complaint: "The shoes I received had a tear on the side."
Category: Product quality

Complaint: "The support agent was rude and didn’t help at all."
Category: Customer service

Complaint: "I had trouble applying the discount code during checkout."
Category: Other

Now classify the following complaint:

Complaint: {complaint}
Category:"""

In [23]:
# Complaint to classify
complaint_to_classify = (
    "The delivery guy just left the package at the gate without ringing the bell. "
    "I found it two hours later completely soaked from the rain."
)

In [24]:
print(few_shot_prompt.format({"complaint": complaint_to_classify[0]}))

Without Chain of Thought:

Classify the following customer complaints into one of these categories:
1. Delivery issue
2. Product quality
3. Customer service
4. Other

Examples:

Complaint: "My order was supposed to arrive yesterday, but it hasn’t shown up yet."
Category: Delivery issue

Complaint: "The shoes I received had a tear on the side."
Category: Product quality

Complaint: "The support agent was rude and didn’t help at all."
Category: Customer service

Complaint: "I had trouble applying the discount code during checkout."
Category: Other

Now classify the following complaint:

Complaint: The delivery guy just left the package at the gate without ringing the bell. I found it two hours later completely soaked from the rain.
Category:


In [28]:
print(get_response_v0(few_shot_prompt.format({"complaint": complaint_to_classify[0]})))

Delivery issue


## Chain of Thought

In [20]:
# Function to call OpenAI
def get_response_v2(system_prompt, prompt):

    print( "**************** Prompt *****************")
    print(system_prompt + prompt)
    print( "****************************************")

    completion = client.chat.completions.create(
      model="gpt-4o",
      messages=[
          {"role": "developer", "content": system_prompt},
          {
              "role": "user",
              "content": prompt,
          },
      ],
    )

    return completion.choices[0].message.content

In [21]:
system_prompt = """Your job is to classify complaints into one of the following categories:

Delivery issue
Product quality
Customer service
Other

Use chain-of-thought reasoning to guide the classification of the following complaint."""

user_prompt = f"""
Complaint: {complaint}
"""

print("With Chain of Thought:")
print(get_response_v2(system_prompt,
                      user_prompt.format({"complaint":
                                         "The delivery guy just left the package at the gate without ringing the bell. I found it two hours later completely soaked from the rain."})))

With Chain of Thought:
To classify this complaint, I'll go through the details provided:

1. The complaint is about how the delivery was handled.
2. The delivery personnel did not follow proper procedure, such as ringing the bell.
3. The package was left in an inappropriate place (at the gate), which led to it being damaged by rain.
4. The core issue revolves around the method and location of delivery which led to the package being in poor condition.

Given these points, the complaint is primarily related to the handling and execution of the delivery process. Therefore, this complaint falls under the category of:

Delivery issue


## Structured Response

In [51]:
# System prompt
system_prompt = "You are a helpful assistant that classifies customer complaints. Return the result only as a JSON object: {\"category\": \"...\"}"

In [52]:
user_prompt = f"""
Classify the customer complaint into one of these categories:
1. Delivery issue
2. Product quality
3. Customer service
4. Other

Complaint: {complaint}
"""

In [53]:
resp = get_response_v2(system_prompt,
                      user_prompt.format({"complaint":
                                         "The delivery guy just left the package at the gate without ringing the bell. I found it two hours later completely soaked from the rain."}))

In [57]:
resp

'{"category": "Delivery issue"}'

In [59]:
import json

category_dict = json.loads(resp)

print(category_dict)
print(category_dict["category"])

{'category': 'Delivery issue'}
Delivery issue


## Temperature

In [68]:
# API wrapper
def get_response_v3(system_prompt, prompt, temperature=0.7):

    print( "**************** Prompt *****************")
    print(system_prompt + prompt)
    print( "****************************************")

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt},
        ],
        temperature=temperature
    )
    return completion.choices[0].message.content.strip()

In [69]:
system_prompt = """Your job is to classify complaints into one of the following categories:

Delivery issue
Product quality
Customer service
Other

Use chain-of-thought reasoning to guide the classification of the following complaint."""

user_prompt = f"""
Complaint: {complaint}
"""

In [46]:
print(get_response_v3(system_prompt,
                      user_prompt.format({"complaint":
                                         "The delivery guy just left the package at the gate without ringing the bell. I found it two hours later completely soaked from the rain."})))

To classify this complaint, let's analyze the details provided:

1. The complaint mentions a "delivery guy," which indicates that the issue is related to the delivery process.
2. The specific problem is that the package was left at the gate without any notification (not ringing the bell), which is a delivery method issue.
3. As a result of the delivery method, the package was exposed to rain and got soaked. This consequence further emphasizes the problem with how the delivery was handled.

Given these points, the primary concern in this complaint revolves around the delivery process and how it was executed. Therefore, this complaint is best classified under the category of "Delivery issue."


In [49]:
print(get_response_v3(system_prompt,
                      user_prompt.format({"complaint":
                                         "The delivery guy just left the package at the gate without ringing the bell. I found it two hours later completely soaked from the rain."}),
                      temperature = 0.0))

To classify this complaint, let's break down the details:

1. The complaint mentions the "delivery guy" and the action of leaving the package at the gate.
2. The issue arises from the delivery person's failure to ring the bell, which is a part of the delivery process.
3. The consequence of this action was that the package was left outside for two hours and got soaked from the rain.

The primary focus of the complaint is on the delivery process and the actions of the delivery person. The problem originated from how the delivery was handled, leading to the package being exposed to rain. Therefore, this complaint falls under the category of a "Delivery issue."


## Self Consistency

In [76]:
import time
from collections import Counter

system_prompt = "You are a helpful assistant that classifies customer complaints. Return the result only as a JSON object: {\"category\": \"...\"}"

user_prompt = f"""
Classify the customer complaint into one of these categories:
1. Delivery issue
2. Product quality
3. Customer service
4. Other

Use chain-of-thought reasoning to guide the classification of the following complaint.

Complaint: {complaint}
"""

complaint = "I waited all day at home, but the package never came. When I contacted support,  they told me to wait another 48 hours without giving a reason."

print(user_prompt.format({"complaint": complaint}))

results = []

for temp in [0.0, 0.5, 2.0]:
    try:
        resp = get_response_v3(system_prompt
                              , user_prompt.format({"complaint": complaint})
                              , temperature = temp)
        category_dict = json.loads(resp)
        category = category_dict['category']
        if category is not None:
            results.append(category)
    except Exception as e:
        print(f"Error on run {temp}: {e}")

    time.sleep(1)  # to avoid rate limits

    counts = Counter(results)
    majority = counts.most_common(1)[0]

print("All predictions:", results)
print("Majority vote:", majority)


Classify the customer complaint into one of these categories:
1. Delivery issue
2. Product quality
3. Customer service
4. Other

Use chain-of-thought reasoning to guide the classification of the following complaint.

Complaint: I waited all day at home, but the package never came. When I contacted support, 
             they told me to wait another 48 hours without giving a reason. Finally, the delivery guys delivered the wrong package

**************** Prompt *****************
You are a helpful assistant that classifies customer complaints. Return the result only as a JSON object: {"category": "..."}
Classify the customer complaint into one of these categories:
1. Delivery issue
2. Product quality
3. Customer service
4. Other

Use chain-of-thought reasoning to guide the classification of the following complaint.

Complaint: I waited all day at home, but the package never came. When I contacted support, 
             they told me to wait another 48 hours without giving a reason. Final